In [53]:
### Log In

import getpass
from subprocess import check_output

print("cloudForge log in")
u = raw_input("username:")
p = getpass.getpass("password:")

# get the API endpoint
print ("get the API endpoint...")
foo = check_output(["cf", "login", "-u", u, "-p", p])
foo = check_output(["cf", "api"])
bar = foo.split()
MY_PR_API_ENDPOINT = bar[2]
print ("MY_PR_API_ENDPOINT " + MY_PR_API_ENDPOINT)

# get the OAUTH bearer token
print ("get the OAUTH bearer token...")
foo = check_output(["cf", "oauth-token"])
bar = foo.split()
MY_PR_BEARER_TOKEN = bar[1]
MY_PR_BEARER_TOKEN_W_AUTH = "'Authorization: bearer " + MY_PR_BEARER_TOKEN + "'"
print ("MY_PR_BEARER_TOKEN " + MY_PR_BEARER_TOKEN)

print("done")

cloudForge log in
username:cwinsor@gmail.com
password:········
get the API endpoint...
MY_PR_API_ENDPOINT https://api.system.aws-usw02-pr.ice.predix.io
get the OAUTH bearer token...
MY_PR_BEARER_TOKEN eyJhbGciOiJSUzI1NiIsImtpZCI6ImxlZ2FjeS10b2tlbi1rZXkiLCJ0eXAiOiJKV1QifQ.eyJqdGkiOiI3YTQ2YTliNGQ0NGM0YWQwOTNkNjk0ZDY3MTI4MGRlMiIsInN1YiI6IjM1NjNmOTE2LThiOWYtNDRhMi1iMTAzLTFiY2RiNmE1ZjJkNiIsInNjb3BlIjpbImNsb3VkX2NvbnRyb2xsZXIucmVhZCIsInBhc3N3b3JkLndyaXRlIiwiY2xvdWRfY29udHJvbGxlci53cml0ZSIsIm9wZW5pZCIsInVhYS51c2VyIl0sImNsaWVudF9pZCI6ImNmIiwiY2lkIjoiY2YiLCJhenAiOiJjZiIsImdyYW50X3R5cGUiOiJwYXNzd29yZCIsInVzZXJfaWQiOiIzNTYzZjkxNi04YjlmLTQ0YTItYjEwMy0xYmNkYjZhNWYyZDYiLCJvcmlnaW4iOiJ1YWEiLCJ1c2VyX25hbWUiOiJjd2luc29yQGdtYWlsLmNvbSIsImVtYWlsIjoiY3dpbnNvckBnbWFpbC5jb20iLCJyZXZfc2lnIjoiNjMyMzE2ZSIsImlhdCI6MTQ4MDQ2ODI5OCwiZXhwIjoxNDgwNDc1NDk4LCJpc3MiOiJodHRwczovL3VhYS5zeXN0ZW0uYXdzLXVzdzAyLXByLmljZS5wcmVkaXguaW8vb2F1dGgvdG9rZW4iLCJ6aWQiOiJ1YWEiLCJhdWQiOlsiY2xvdWRfY29udHJvbGxlciIsInBhc3N3b3JkIiwiY2YiLCJ1

In [58]:
# Identify what space to target.  Get the guig

# cf curl /v2/spaces
foo = check_output(["cf", "curl", "/v2/spaces"])
import json
blah = json.loads(foo)
#print json.dumps(blah, sort_keys=True, indent=4, separators=(',', ': '))

dict = {}
for num, line in enumerate(blah["resources"]):
    dict[num] = line["entity"]["name"]

if (len(dict)==1):
    sp = 0;
else:
    finished = False
    while not finished:
        sp = input("Choose the space you want to target:\n     " + str(dict))
        try:
            t = blah["resources"][sp]
            finished = True
        except IndexError:
            finished = False

MY_PR_GUID   = blah["resources"][sp]["metadata"]["guid"]
MY_PR_URL    = blah["resources"][sp]["metadata"]["url"]
MY_PR_NAME   = blah["resources"][sp]["entity"]["name"]
MY_PR_ENTITY = blah["resources"][sp]["entity"]

lst = ["cf", "target", "-s", MY_PR_NAME]
foo = check_output(lst)

print "targeting space: \"" + MY_PR_NAME + "\""
print "url: " + MY_PR_URL

Choose the space you want to target:
     {0: u'dev', 1: u'other'}0
targeting space: "dev"
url: /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc


In [57]:
### Create UAA Service
### cf create-service predix-uaa Free my-uaa -c '{"adminClientSecret":"my-secret"}'

print("Create UAA Service")
secret = getpass.getpass('secret code:')
aa = "\"adminClientSecret\""
bb = "\"" + secret + "\""
cc = "{" + aa + ":" + bb + "}"
#print cc
dd = ["cf", "create-service", "predix-uaa", "Free", "my-uaa", "-c", cc]
#print dd
foo = check_output(dd)
print foo

print("done")

Create UAA Service
secret code:········
Creating service instance my-uaa in org cwinsor@gmail.com / space dev as cwinsor@gmail.com...
OK

done


In [59]:
### choose the UAA service instance and get its details
from subprocess import check_output

lst = ["curl", "-H", "Authorization: bearer " + MY_PR_BEARER_TOKEN,  MY_PR_API_ENDPOINT + MY_PR_URL + "/summary"]
foo = check_output(lst)
import json
blah = json.loads(foo)
#print json.dumps(blah, sort_keys=True, indent=4, separators=(',', ': '))

dict = {}
for num, line in enumerate(blah["services"]) :
    if (line["service_plan"]["service"]["label"] == "predix-uaa"):
        dict[num] = line["name"]

if (len(dict)==1):
    sp = 0;
else:
    finished = False
    while not finished:
        sp = input("Choose the UAA service you want to use:\n     " + str(dict))
        try:
            t = blah["services"][sp]
            finished = True
        except IndexError:
            finished = False
    
MY_PR_UAA_GUID   =  blah["services"][sp]["guid"]
MY_PR_UAA_NAME   =  blah["services"][sp]["name"]

print "using \"" + MY_PR_UAA_NAME + "\""
print "GUID: " + MY_PR_UAA_GUID

using "my-uaa"
GUID: f1cfab72-7cf4-4b16-8956-25cf9aa0362c


In [60]:
### Create Asset Service
### cf create-service predix-asset Tiered my-asset -c '{"trustedIssuerIds":["https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token"]}'

print("Create Asset Service")
issuer_id = json.dumps({"trustedIssuerIds": [MY_PR_API_ENDPOINT + MY_PR_URL + "/oauth/token"]})
dd = ["cf", "create-service", "predix-asset", "Tiered", "my-asset", "-c", issuer_id]
foo = check_output(dd)
print("done")

Create Asset Service
done


In [61]:
### choose the Asset service instance and get its details
from subprocess import check_output

lst = ["curl", "-H", "Authorization: bearer " + MY_PR_BEARER_TOKEN,  MY_PR_API_ENDPOINT + MY_PR_URL + "/summary"]
foo = check_output(lst)
print foo

import json
blah = json.loads(foo)
#print json.dumps(blah, sort_keys=True, indent=4, separators=(',', ': '))

dict = {}
for num, line in enumerate(blah["services"]) :
    if (line["service_plan"]["service"]["label"] == "predix-asset"):
        dict[num] = line["name"]

if (len(dict)==1):
    sp = 0;
else:
    finished = False
    while not finished:
        sp = input("Choose the Asset service you want to use:\n     " + str(dict))
        try:
            t = blah["services"][sp]
            finished = True
        except IndexError:
            finished = False
    
MY_PR_ASSET_GUID   =  blah["services"][sp]["guid"]
MY_PR_ASSET_NAME   =  blah["services"][sp]["name"]

print "using \"" + MY_PR_UAA_NAME + "\""
print "GUID: " + MY_PR_UAA_GUID

{
  "guid": "f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc",
  "name": "dev",
  "apps": [

  ],
  "services": [
    {
      "guid": "22f7c171-2e1b-4b10-9812-091feca9d1ff",
      "name": "my-asset",
      "bound_app_count": 0,
      "last_operation": {
        "type": "create",
        "state": "succeeded",
        "description": "",
        "updated_at": null,
        "created_at": "2016-11-30T01:20:31Z"
      },
      "dashboard_url": null,
      "service_plan": {
        "guid": "e08d80b5-6da3-4f29-a2f1-f7834d06cd29",
        "name": "Tiered",
        "service": {
          "guid": "90fc045c-adb0-41c6-9c5d-072d3d2c44b2",
          "label": "predix-asset",
          "provider": null,
          "version": null
        }
      }
    },
    {
      "guid": "f1cfab72-7cf4-4b16-8956-25cf9aa0362c",
      "name": "my-uaa",
      "bound_app_count": 0,
      "last_operation": {
        "type": "create",
        "state": "succeeded",
        "description": "",
        "updated_at": null,
        "create